# Navier Stokes - blood flow simulation

## Formulation

In this example we present a bloodflow simulation in a 3D geometry and we demonstrate how to prescribe a time dependent expression as a Dirichlet boundary condition.
![Mesh](fig/aneurysm.png "Mesh")

## Implementation
Again, we need to import the neccessary packages.

In [94]:
import dolfin as df
import numpy as np
from time import time

Next, we read the mesh from the file and create the boundary markings since ```.xml``` files do not include them.

In [95]:
mesh = df.Mesh("bifurcation_murray_mesh.xml")
dim = mesh.topology().dim()
bndry = df.MeshFunction("size_t", mesh, dim-1, 0)
mesh.init()

points = [np.array([-0.9, 0.0, 0.0]), np.array([0.6, 0.0, 0.6]), np.array([0.6, 0.0, -0.6])]
normals = [np.array([-1.0, 0.0, 0.0]), np.array([np.sqrt(2), 0.0, np.sqrt(2)]), np.array([np.sqrt(2), 0.0, -np.sqrt(2)])]
for f in df.facets(mesh):
    bndry[f] = 0
    if f.exterior():
        bndry[f] = 1
        x = f.midpoint().array()[:dim]
        for mark, point, normal in zip([2,3,4], points, normals):
            if abs(np.dot(x-point, normal)) <= 0.001:
                bndry[f] = mark

Now we define the function spaces for MINI element.

In [96]:
U = df.FiniteElement("CG", mesh.ufl_cell(), 1)
B = df.FiniteElement("Bubble", mesh.ufl_cell(), dim + 1)
P = df.FiniteElement("CG", mesh.ufl_cell(), 1)
V = df.VectorElement(df.NodalEnrichedElement(U, B))
W = df.FunctionSpace(mesh, df.MixedElement([V, P]))

Let us define some parameters and constants.

In [1]:
mu = df.Constant(0.04)
rho = df.Constant(1.05)
theta = df.Constant(1.0)
dt = 0.01
t_end = 0.35
t = 0.0

NameError: name 'df' is not defined

Then we can define the boundary conditions as no slip on the wall and parabolic expression at the inlet. Notice that the average inflow velocity is now prescribed using a function of time. 

In [98]:
zero_vector = df.Constant((0.0, 0.0, 0.0))
bc_walls = df.DirichletBC(W.sub(0), zero_vector, bndry, 1)

velocity = lambda t: 0.6*max((t*(0.3-t))/pow(0.15,2), 0.0)
inlet_expression = df.Expression(("v * 2.0 * (pow(R,2) - pow(x[0]+0.9,2) - pow(x[1],2) - pow(x[2],2)) / pow(R,2)", "0.0", "0.0"), R=0.15, v=velocity(t), degree=2)
bc_in = df.DirichletBC(W.sub(0), inlet_expression, bndry, 2)

bcs = [bc_in, bc_walls]

Then we can define the variational problem.

In [100]:
v, q = df.TestFunctions(W)
w = df.Function(W)
w0 = df.Function(W)
u, p = df.split(w)
u0, p0 = df.split(w0)

a = lambda u, v: rho*df.inner(df.grad(u)*u, v)*df.dx + mu*df.inner(df.grad(u), df.grad(v))*df.dx
b = lambda q, v: q*df.div(v)*df.dx

F1 = a(u, v) - b(p, v) - b(q, u)
F0 = a(u0, v) - b(p, v) - b(q, u)

# time derivative
F = df.Constant(1.0/dt)*rho*df.inner((u-u0), v)*df.dx + theta*F1 + (1-theta)*F0

# Jacobi matrix
J = df.derivative(F, w)

Setting up nonlinear variational solver.

In [101]:
problem = df.NonlinearVariationalProblem(F, w, bcs, J)
solver = df.NonlinearVariationalSolver(problem)
solver.parameters['newton_solver']['linear_solver'] = 'mumps'
solver.parameters['newton_solver']['absolute_tolerance'] = 1e-12
solver.parameters['newton_solver']['relative_tolerance'] = 1e-12

Setup files to save results for viewing in Paraview.

In [102]:
ufile = df.XDMFFile("results/u.xdmf")
pfile = df.XDMFFile("results/p.xdmf")
ufile.parameters["flush_output"] = True
pfile.parameters["flush_output"] = True

TODO: Time stepping

In [103]:
tick = time()
t = 0.0
(u, p) = w.split(True)
u.rename("v", "velocity")
p.rename("p", "pressure")
df.assign(u, w.sub(0))
df.assign(p, w.sub(1))
ufile.write(u, t)
pfile.write(p, t)
while t < t_end:
    w0.assign(w)
    t += dt
    print("t={:.2f}s".format(t))
    inlet_expression.v = velocity(t)
    solver.solve()
    df.assign(u, w.sub(0))
    df.assign(p, w.sub(1))
    ufile.write(u, t)
    pfile.write(p, t)
print("ellapsed = ", time() - tick, "s")

t=0.01s
  Solving nonlinear variational problem.
    Newton iteration 0: r (abs) = 1.038e+01 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
    Newton iteration 1: r (abs) = 2.291e-02 (tol = 1.000e-12) r (rel) = 2.207e-03 (tol = 1.000e-12)
    Newton iteration 2: r (abs) = 6.536e-06 (tol = 1.000e-12) r (rel) = 6.295e-07 (tol = 1.000e-12)
    Newton iteration 3: r (abs) = 1.223e-12 (tol = 1.000e-12) r (rel) = 1.178e-13 (tol = 1.000e-12)
    Newton solver finished in 3 iterations and 3 linear solver iterations.
t=0.02s
  Solving nonlinear variational problem.
    Newton iteration 0: r (abs) = 2.096e+01 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
    Newton iteration 1: r (abs) = 2.004e-02 (tol = 1.000e-12) r (rel) = 9.560e-04 (tol = 1.000e-12)
    Newton iteration 2: r (abs) = 4.787e-06 (tol = 1.000e-12) r (rel) = 2.284e-07 (tol = 1.000e-12)
    Newton iteration 3: r (abs) = 9.672e-13 (tol = 1.000e-12) r (rel) = 4.614e-14 (tol = 1.000e-12)
    Newton solver finished 

t=0.17s
  Solving nonlinear variational problem.
    Newton iteration 0: r (abs) = 1.969e+00 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
    Newton iteration 1: r (abs) = 5.602e-03 (tol = 1.000e-12) r (rel) = 2.845e-03 (tol = 1.000e-12)
    Newton iteration 2: r (abs) = 1.290e-07 (tol = 1.000e-12) r (rel) = 6.553e-08 (tol = 1.000e-12)
    Newton iteration 3: r (abs) = 7.924e-14 (tol = 1.000e-12) r (rel) = 4.024e-14 (tol = 1.000e-12)
    Newton solver finished in 3 iterations and 3 linear solver iterations.
t=0.18s
  Solving nonlinear variational problem.
t=0.19s
    Newton iteration 0: r (abs) = 2.998e+00 (tol = 1.000e-12) r (rel) = 1.000e+00 (tol = 1.000e-12)
    Newton iteration 1: r (abs) = 6.449e-03 (tol = 1.000e-12) r (rel) = 2.151e-03 (tol = 1.000e-12)
    Newton iteration 2: r (abs) = 1.491e-07 (tol = 1.000e-12) r (rel) = 4.974e-08 (tol = 1.000e-12)
    Newton iteration 3: r (abs) = 1.461e-13 (tol = 1.000e-12) r (rel) = 4.874e-14 (tol = 1.000e-12)
    Newton solver f